In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
from collections import deque
import pandas as pd
import cv2


input_dim = 5  # one-hot size
hidden_size = 4  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 5  # |ihello| == 6
learning_rate = 0.0001
last5_img = deque(maxlen=5)
last5_keyout = deque(maxlen=5)
data_mean = 17.119790415671556
data_std = 63.36037302373175

In [2]:
X = tf.placeholder(tf.float32, [5, 120, 300, 1])
Y = tf.placeholder(tf.float32, [5, hidden_size])

with tf.variable_scope("cnn"):
    
    # conv : 5, 200, 400, 16
    conv = tf.layers.conv2d(inputs=X, filters=16, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool : 5, 50, 99, 16
    pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 4], padding='SAME', strides=1)
    # conv2 
    conv2 = tf.layers.conv2d(inputs=X, filters=64, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool2 
    pool2 = tf.layers.max_pooling2d(inputs=conv, pool_size=[4, 8], padding='SAME', strides=2)
    # conv3
    conv2 = tf.layers.conv2d(inputs=X, filters=128, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool3 
    pool3 = tf.layers.max_pooling2d(inputs=conv, pool_size=[4, 8], padding='SAME', strides=2)
    
    pool3 = tf.reshape(pool, [1, 5, -1])

In [3]:
with tf.variable_scope("rnn"):
    cell = tf.contrib.rnn.LSTMCell(num_units=hidden_size, state_is_tuple=True)
    
    initial_state = cell.zero_state(batch_size, tf.float32)
    #outputs : 1, 5, 50*99*16
    outputs, _states = tf.nn.dynamic_rnn(cell, pool3, initial_state=initial_state, dtype=tf.float32)
    
    X_for_sigmoid = tf.reshape(outputs, [-1, hidden_size])
    
    sigmoid_w = tf.get_variable("softmax_w", [hidden_size, hidden_size])
    sigmoid_b = tf.get_variable("softmax_b", [hidden_size])
    outputs = tf.sigmoid(tf.matmul(X_for_sigmoid, sigmoid_w) + sigmoid_b)
    
    loss = -tf.reduce_mean(Y*tf.log(outputs)+(1-Y)*(tf.log(1-outputs)))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [4]:
saver = tf.train.Saver()

with tf.Session() as sess:
    res = raw_input("Do you want addtional learning? (Name or Enter)")
    if res:
        saver.restore(sess, "./model/"+res+".ckpt")
    else:
        sess.run(tf.global_variables_initializer())
    model_name = raw_input("Model_Name: ")
    
    try:
        for epoch in range(4):

            data_df = pd.read_csv('log.csv', names=['name','key_out'])
            names = data_df['name'].values
            name_i = 0
            # keyouts = [8, 1, 4, 2, 15, ...]
            key_out = [eval(x) for x in data_df['key_out'].values]

            while len(last5_img) < 4 and len(names)>4:
                img = Image.open("./img/"+names[name_i])
                data = np.array( img, dtype='uint8' )
                data = (data-data_mean) / data_std
                last5_img.append(data)
                last5_keyout.append(key_out[name_i])
                name_i+=1

            while(name_i < len(names)):
                img = Image.open("./img/"+names[name_i])
                data = np.array( img, dtype='uint8')
                last5_img.append(data)
                x_data = np.reshape(last5_img, [5, 120, 300, 1])
                last5_keyout.append(key_out[name_i])

                print name_i, "/", len(names), sess.run(loss, feed_dict={X:np.array(x_data, np.float32), Y:last5_keyout})
                sess.run(train, feed_dict={X:np.array(x_data, np.float32), Y:last5_keyout})
                name_i += 1
            print 'epoch', epoch+1
    finally:
        save_path = saver.save(sess, "./model/"+model_name+".ckpt")

Do you want addtional learning? (Name or Enter)v6_1
INFO:tensorflow:Restoring parameters from ./model/v6_1.ckpt
Model_Name: v6)5
4 / 1954 0.4199771
5 / 1954 0.41949996
6 / 1954 0.4667302
7 / 1954 0.46588737
8 / 1954 0.4604321
9 / 1954 0.41839713
10 / 1954 0.41289988
11 / 1954 0.3717034
12 / 1954 0.37159055
13 / 1954 0.37147874
14 / 1954 0.37136775
15 / 1954 0.37125745
16 / 1954 0.37114772
17 / 1954 0.40795627
18 / 1954 0.46901876
19 / 1954 0.5299077
20 / 1954 0.5651221
21 / 1954 0.5906286
22 / 1954 0.5384477
23 / 1954 0.5165776
24 / 1954 0.53220284
25 / 1954 0.55755347
26 / 1954 0.5649103
27 / 1954 0.5534668
28 / 1954 0.5383507
29 / 1954 0.47948128
30 / 1954 0.47181433
31 / 1954 0.5039119
32 / 1954 0.49584216
33 / 1954 0.46622387
34 / 1954 0.5451845
35 / 1954 0.5576315
36 / 1954 0.58158475
37 / 1954 0.63875216
38 / 1954 0.64908713
39 / 1954 0.67060184
40 / 1954 0.6707943
41 / 1954 0.6725596
42 / 1954 0.6847156
43 / 1954 0.73808783
44 / 1954 0.6786223
45 / 1954 0.6502517
46 / 1954 0.625

383 / 1954 0.426112
384 / 1954 0.39061922
385 / 1954 0.4279859
386 / 1954 0.45850706
387 / 1954 0.46768475
388 / 1954 0.504883
389 / 1954 0.53400946
390 / 1954 0.54857635
391 / 1954 0.5693675
392 / 1954 0.6434712
393 / 1954 0.6433934
394 / 1954 0.6427464
395 / 1954 0.6105261
396 / 1954 0.5855429
397 / 1954 0.4948667
398 / 1954 0.45626372
399 / 1954 0.47731605
400 / 1954 0.44009596
401 / 1954 0.44900465
402 / 1954 0.4457921
403 / 1954 0.42503768
404 / 1954 0.38848108
405 / 1954 0.38046724
406 / 1954 0.3513009
407 / 1954 0.3512446
408 / 1954 0.40468064
409 / 1954 0.44310826
410 / 1954 0.5024226
411 / 1954 0.5547308
412 / 1954 0.5852263
413 / 1954 0.5278065
414 / 1954 0.515464
415 / 1954 0.511034
416 / 1954 0.4952115
417 / 1954 0.50312173
418 / 1954 0.53218544
419 / 1954 0.55318546
420 / 1954 0.51458883
421 / 1954 0.514283
422 / 1954 0.533608
423 / 1954 0.4836906
424 / 1954 0.44658643
425 / 1954 0.43543673
426 / 1954 0.38576627
427 / 1954 0.35029826
428 / 1954 0.40988714
429 / 1954 0.4097

763 / 1954 0.47961932
764 / 1954 0.43906474
765 / 1954 0.39871806
766 / 1954 0.35966995
767 / 1954 0.41044587
768 / 1954 0.41037974
769 / 1954 0.4100875
770 / 1954 0.47043926
771 / 1954 0.52033377
772 / 1954 0.45272726
773 / 1954 0.4490398
774 / 1954 0.4248205
775 / 1954 0.46970344
776 / 1954 0.49428588
777 / 1954 0.53407395
778 / 1954 0.57015836
779 / 1954 0.58210933
780 / 1954 0.588281
781 / 1954 0.6031436
782 / 1954 0.6244841
783 / 1954 0.68319196
784 / 1954 0.68174374
785 / 1954 0.64794433
786 / 1954 0.5844114
787 / 1954 0.55462384
788 / 1954 0.5237459
789 / 1954 0.47879428
790 / 1954 0.4263907
791 / 1954 0.4574015
792 / 1954 0.41947025
793 / 1954 0.36827052
794 / 1954 0.40735254
795 / 1954 0.43942347
796 / 1954 0.44889385
797 / 1954 0.48775688
798 / 1954 0.51835334
799 / 1954 0.47791743
800 / 1954 0.43752223
801 / 1954 0.39732486
802 / 1954 0.3584146
803 / 1954 0.32776025
804 / 1954 0.32771662
805 / 1954 0.32766747
806 / 1954 0.3276134
807 / 1954 0.32755488
808 / 1954 0.3274924
80

1137 / 1954 0.42610392
1138 / 1954 0.42624307
1139 / 1954 0.38758987
1140 / 1954 0.39701128
1141 / 1954 0.395681
1142 / 1954 0.44898778
1143 / 1954 0.41729435
1144 / 1954 0.44859523
1145 / 1954 0.4549592
1146 / 1954 0.47377872
1147 / 1954 0.4968539
1148 / 1954 0.5488869
1149 / 1954 0.57037616
1150 / 1954 0.5899387
1151 / 1954 0.5350398
1152 / 1954 0.5131846
1153 / 1954 0.5158128
1154 / 1954 0.49743518
1155 / 1954 0.45882225
1156 / 1954 0.44924802
1157 / 1954 0.4098189
1158 / 1954 0.3469221
1159 / 1954 0.37835735
1160 / 1954 0.37825844
1161 / 1954 0.43949515
1162 / 1954 0.4361894
1163 / 1954 0.4148218
1164 / 1954 0.43614587
1165 / 1954 0.47686797
1166 / 1954 0.43916863
1167 / 1954 0.47537956
1168 / 1954 0.45123115
1169 / 1954 0.39301515
1170 / 1954 0.45873803
1171 / 1954 0.45049208
1172 / 1954 0.419802
1173 / 1954 0.48153442
1174 / 1954 0.4653625
1175 / 1954 0.37992018
1176 / 1954 0.44035253
1177 / 1954 0.43230692
1178 / 1954 0.37690675
1179 / 1954 0.41368324
1180 / 1954 0.43278313
1181

1501 / 1954 0.49328765
1502 / 1954 0.49322945
1503 / 1954 0.5138252
1504 / 1954 0.53440136
1505 / 1954 0.49479073
1506 / 1954 0.49258596
1507 / 1954 0.4390359
1508 / 1954 0.38255858
1509 / 1954 0.34562492
1510 / 1954 0.33753118
1511 / 1954 0.30780607
1512 / 1954 0.30777335
1513 / 1954 0.30773467
1514 / 1954 0.3076905
1515 / 1954 0.3675693
1516 / 1954 0.3674632
1517 / 1954 0.43552828
1518 / 1954 0.43231067
1519 / 1954 0.41148773
1520 / 1954 0.3744371
1521 / 1954 0.36578673
1522 / 1954 0.3072539
1523 / 1954 0.30719122
1524 / 1954 0.30712557
1525 / 1954 0.34634995
1526 / 1954 0.34627214
1527 / 1954 0.40605325
1528 / 1954 0.40470967
1529 / 1954 0.45616692
1530 / 1954 0.4831082
1531 / 1954 0.5013155
1532 / 1954 0.4618888
1533 / 1954 0.4772895
1534 / 1954 0.41960734
1535 / 1954 0.3755375
1536 / 1954 0.40441266
1537 / 1954 0.3962095
1538 / 1954 0.43468642
1539 / 1954 0.49151117
1540 / 1954 0.47061348
1541 / 1954 0.5019382
1542 / 1954 0.49007207
1543 / 1954 0.44999593
1544 / 1954 0.412845
1545

1866 / 1954 0.2993764
1867 / 1954 0.29933006
1868 / 1954 0.37234405
1869 / 1954 0.37227458
1870 / 1954 0.37203273
1871 / 1954 0.44365197
1872 / 1954 0.47373772
1873 / 1954 0.450776
1874 / 1954 0.44913286
1875 / 1954 0.4384048
1876 / 1954 0.36685348
1877 / 1954 0.32876056
1878 / 1954 0.29876208
1879 / 1954 0.29870802
1880 / 1954 0.3381722
1881 / 1954 0.41123605
1882 / 1954 0.48414135
1883 / 1954 0.55578697
1884 / 1954 0.58553827
1885 / 1954 0.54455125
1886 / 1954 0.5109848
1887 / 1954 0.43819198
1888 / 1954 0.36767626
1889 / 1954 0.33637244
1890 / 1954 0.32820916
1891 / 1954 0.3558578
1892 / 1954 0.41350317
1893 / 1954 0.47074017
1894 / 1954 0.52488047
1895 / 1954 0.5587478
1896 / 1954 0.54058504
1897 / 1954 0.48281938
1898 / 1954 0.46500802
1899 / 1954 0.40955752
1900 / 1954 0.36734194
1901 / 1954 0.4668889
1902 / 1954 0.5318455
1903 / 1954 0.54076284
1904 / 1954 0.66693956
1905 / 1954 0.69339246
1906 / 1954 0.6257178
1907 / 1954 0.6080297
1908 / 1954 0.6026696
1909 / 1954 0.54223317
1

KeyboardInterrupt: 